# Duckdb example

Read the data with duckdb and example

SQL and Python are 2 of the top 10 [programming language of the world](https://www.tiobe.com/tiobe-index/) ;)


In [12]:
import futuredata as fd

In [13]:
duck_engine = fd.DuckEngine(database="duckdb", read_only=True)
conn = duck_engine.get_connection()

In [14]:
# configure the settings
conn.execute(
    f"""
        INSTALL httpfs;
        LOAD httpfs;
        SET s3_region = '{fd.settings.AWS_REGION}';
        SET s3_access_key_id = '{fd.settings.AWS_ACCESS_KEY_ID}';
        SET s3_secret_access_key = '{fd.settings.AWS_SECRET_ACCESS_KEY}';
    """
)

In [37]:
dataset = conn.read_parquet("s3://intella/test/text/article_fact/*/*")

In [45]:
local_dataset = conn.read_parquet("~/Downloads/mdpi/article_fact/*/*")

In [46]:
# materialize the dataframe using polars
df = local_dataset.pl()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [47]:
type(df)

polars.dataframe.frame.DataFrame

In [38]:
dataset.columns

['article_id',
 'article_doi_id',
 'article_pii_id',
 'article_pmid_id',
 'journal_id',
 'section_id',
 'special_issue_id',
 'article_type_id',
 'article_type_name',
 'submission_id',
 'copyrights_id',
 'manuscript_id',
 'reference_batch_id',
 'article_publisher',
 'article_volume',
 'article_year',
 'article_issue',
 'article_number',
 'article_lastpage',
 'article_pubdate_published',
 'article_pubdate_received',
 'article_pubdate_revised',
 'article_pubdate_accepted',
 'article_date_inserted',
 'article_date_updated',
 'article_date_published',
 'article_title',
 'article_abstract',
 'article_keywords',
 'article_reference_citeby_number',
 'article_reference_citeby_number_max',
 'article_citations_last_update',
 'article_check_sum',
 'article_is_public',
 'submission_hash_key',
 'submission_status_id',
 'submission_number_authors',
 'ai_ready']

In [41]:
dataset.query("test", "select article_title, article_abstract from dataset")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌──────────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────┐
│    article_title     │                                       article_abstract                                        │
│       varchar        │                                            varchar                                            │
├──────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────────┤
│ Chemistry of Nitro…  │ The 1-methyl-2-quinolone (MeQone) framework is often found in alkaloids and recently attent…  │
│ Pre-Ischemic Tread…  │ Physical exercise has been shown to be beneficial in stroke patients and animal stroke mode…  │
│ (Z)-2-(4-Chloro-5H…  │ 2-Amino-6-ethoxy-4-phenylpyridine-3,5-dicarbonitrile 1 reacts with 4,5-dichloro-1,2,3-dithi…  │
│ 3-Amino-6-ethoxy-4…  │ (Z)-2-(4-Chloro-5H-1,2,3-dithiazol-5-ylideneamino)-6-ethoxy-4-phenylpyridine-3,5-dicarbonit…  │
│ The More the Worse…  │ Tinnitu

In [49]:
# be carefull because remote_path should be a list
remote_path = ["s3://intella/test/text/article_fact/*/*"]
query = f"select * from read_parquet({remote_path})"
conn.sql(query)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌────────────┬──────────────────────┬────────────────────┬───┬──────────────────────┬──────────────────────┬──────────┐
│ article_id │    article_doi_id    │   article_pii_id   │ … │ submission_status_id │ submission_number_…  │ ai_ready │
│   int64    │       varchar        │      varchar       │   │        int64         │        int64         │ boolean  │
├────────────┼──────────────────────┼────────────────────┼───┼──────────────────────┼──────────────────────┼──────────┤
│       9136 │ 10.3390/molecules1…  │ molecules15085174  │ … │                   29 │                    1 │ true     │
│       9145 │ 10.3390/molecules1…  │ molecules15085246  │ … │                   29 │                    4 │ true     │
│       9156 │ 10.3390/M690         │ M690               │ … │                  102 │                    2 │ true     │
│       9162 │ 10.3390/M691         │ M691               │ … │                  102 │                    2 │ true     │
│       9171 │ 10.3390/ijerph7083…  │ ij